In [1]:
from mbientlab.metawear import MetaWear, libmetawear
from ctypes import c_void_p, cast, POINTER
from time import sleep, time
import sys
import csv

# Import the MetaWear C++ bindings for Python
from mbientlab.metawear.cbindings import *

In [2]:
# Handler for data received from the MetaWear sensors
def data_handler(ctx, data):
    if data.contents.type_id == DataTypeId.CARTESIAN_FLOAT:
        value = cast(data.contents.value, POINTER(CartesianFloat)).contents
        print("X: %.2f Y: %.2f Z: %.2f" % (value.x, value.y, value.z))
        acc_data.append([value.x, value.y, value.z])


In [4]:
# Replace 'XX:XX:XX:XX:XX:XX' with your device's MAC address
device = MetaWear('D4:06:BD:85:8F:23')
device.connect()

In [5]:
# Create a pattern
pattern= LedPattern(repeat_count= Const.LED_REPEAT_INDEFINITELY)

# Load the LED with the pattern
libmetawear.mbl_mw_led_load_preset_pattern(byref(pattern), LedPreset.SOLID)
libmetawear.mbl_mw_led_write_pattern(device.board, byref(pattern), LedColor.GREEN)
# Play the pattern
libmetawear.mbl_mw_led_play(device.board)
sleep(5)
# Stop and clear the pattern from memory
libmetawear.mbl_mw_led_stop_and_clear(device.board)

In [ ]:

acc_data = []
gyro_data = []

# Accelerometer setup
libmetawear.mbl_mw_acc_set_range(device.board, 16.0)  # +/- 16g
libmetawear.mbl_mw_acc_set_odr(device.board, 100.0)  # 100 Hz
libmetawear.mbl_mw_acc_write_acceleration_config(device.board)
acc_signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)

# Gyroscope setup
gyro_signal = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(device.board)
# libmetawear.mbl_mw_gyro_bmi160_set_range(device.board, GyroBmi160Range._2000dps)  # +/- 2000 degrees per second
# libmetawear.mbl_mw_gyro_bmi160_set_odr(device.board, GyroBmi160Odr._100Hz)  # 100 Hz
libmetawear.mbl_mw_gyro_bmi160_write_config(device.board)

# Subscribe to data
libmetawear.mbl_mw_datasignal_subscribe(acc_signal, None, FnVoid_VoidP_DataP(data_handler))
libmetawear.mbl_mw_datasignal_subscribe(gyro_signal, None, FnVoid_VoidP_DataP(data_handler))

# Start sensors
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_acc_start(device.board)
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(device.board)
libmetawear.mbl_mw_gyro_bmi160_start(device.board)

# Collect data for 10 seconds
sleep(10)

# Stop sensors
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_gyro_bmi160_stop(device.board)
libmetawear.mbl_mw_gyro_bmi160_disable_rotation_sampling(device.board)
libmetawear.mbl_mw_datasignal_unsubscribe(acc_signal)
libmetawear.mbl_mw_datasignal_unsubscribe(gyro_signal)

# Disconnec
device.disconnect()

# Writing data to a CSV file
with open('sensor_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Timestamp', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z'])

    # Since timestamps won't perfectly align, we use the closest match for simplicity
    for acc in acc_data:
        # Find closest timestamp in gyro_data
        closest_gyro = min(gyro_data, key=lambda g: abs(g[0] - acc[0]))
        writer.writerow(acc + closest_gyro[1:])  # Append gyro data excluding its own timestamp

print("Data collection complete and saved to 'sensor_data.csv'")


In [1]:
device.on_disconnect = lambda status: print ("we are disconnected!")
device.disconnect()

NameError: name 'device' is not defined